In [ ]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import pandas as pd
from sqlalchemy import delete, func, select

from fund_lens_etl.clients.fec import FECAPIClient
from fund_lens_etl.database import get_db_session
from fund_lens_etl.extractors.fec import (
    FECCandidateExtractor,
    FECCommitteeExtractor,
    FECScheduleAExtractor,
)
from fund_lens_etl.loaders.bronze import (
    BronzeFECCandidateLoader,
    BronzeFECCommitteeLoader,
    BronzeFECScheduleALoader,
)
from fund_lens_etl.models.bronze.fec import (
    BronzeFECCandidate,
    BronzeFECCommittee,
    BronzeFECScheduleA,
)
from fund_lens_etl.models.gold import (
    GoldCandidate,
    GoldCommittee,
    GoldContribution,
    GoldContributor,
)
from fund_lens_etl.models.silver.fec import SilverFECContribution
from fund_lens_etl.transformers.bronze_to_silver import BronzeToSilverFECTransformer
from fund_lens_etl.transformers.silver_to_gold import SilverToGoldFECTransformer
from fund_lens_etl.config import USState

print("✓ All imports successful")
print("\nThis notebook runs the COMPLETE end-to-end pipeline:")
print("  1. Truncate all tables (bronze, silver, gold)")
print("  2. Extract & Load to Bronze (committees, candidates, contributions)")
print("  3. Transform Bronze → Silver (with enrichment)")
print("  4. Transform Silver → Gold (with entity resolution)")

In [ ]:
# Cell 2: Truncate All Tables
print("="*60)
print("STEP 1: Truncate All Tables")
print("="*60)

with get_db_session() as session:
    # Gold layer (in reverse dependency order)
    session.execute(delete(GoldContribution))
    session.execute(delete(GoldContributor))
    session.execute(delete(GoldCommittee))
    session.execute(delete(GoldCandidate))

    # Silver layer
    session.execute(delete(SilverFECContribution))

    # Bronze layer
    session.execute(delete(BronzeFECScheduleA))
    session.execute(delete(BronzeFECCommittee))
    session.execute(delete(BronzeFECCandidate))

    session.commit()

print("✓ All tables truncated")

In [ ]:
# Cell 3: Extract & Load Bronze - Committees
print("="*60)
print("STEP 2a: Extract & Load Committees to Bronze")
print("="*60)

client = FECAPIClient()
committee_extractor = FECCommitteeExtractor(api_client=client)
committee_loader = BronzeFECCommitteeLoader()

committee_df = committee_extractor.extract(
    state=USState.MD,
    cycle=2026,
)

print(f"Extracted {len(committee_df)} committees")

with get_db_session() as session:
    loaded = committee_loader.load(session, committee_df)
    print(f"✓ Loaded {loaded} committees to bronze")

In [ ]:
# Cell 4: Extract & Load Bronze - Candidates
print("="*60)
print("STEP 2b: Extract & Load Candidates to Bronze")
print("="*60)

candidate_extractor = FECCandidateExtractor(api_client=client)
candidate_loader = BronzeFECCandidateLoader()

candidate_df = candidate_extractor.extract(
    state=USState.MD,
    cycle=2026,
)

print(f"Extracted {len(candidate_df)} candidates")

with get_db_session() as session:
    loaded = candidate_loader.load(session, candidate_df)
    print(f"✓ Loaded {loaded} candidates to bronze")

In [ ]:
# Cell 5: Extract & Load Bronze - Contributions (3 committees)
print("="*60)
print("STEP 2c: Extract & Load Contributions to Bronze")
print("="*60)

schedule_a_extractor = FECScheduleAExtractor(api_client=client)
schedule_a_loader = BronzeFECScheduleALoader()

# Get candidate committees
committees = schedule_a_extractor.get_candidate_committees(
    state=USState.MD,
    election_cycle=2026
)

print(f"Found {len(committees)} MD candidate committees")

# Extract contributions from first 3 committees with data
total_loaded = 0
committees_processed = 0
target_committees = 3

for committee in committees:
    if committees_processed >= target_committees:
        break

    print(f"\n{committees_processed + 1}. {committee['committee_name']}")

    all_contributions = []

    try:
        for page_df, metadata in schedule_a_extractor.extract_schedule_a_pages(
            committee_id=committee['committee_id'],
            election_cycle=2026,
            starting_page=1
        ):
            if metadata['page'] == 1 and metadata['total_count'] == 0:
                print(f"   ⊘ No contributions, skipping...")
                break

            all_contributions.append(page_df)

            if sum(len(df) for df in all_contributions) >= 200:
                print(f"   Reached 200 record limit")
                break

        if all_contributions:
            contributions_df = pd.concat(all_contributions, ignore_index=True)
            print(f"   ✓ Extracted {len(contributions_df)} contributions")

            with get_db_session() as session:
                loaded = schedule_a_loader.load(session, contributions_df)
                total_loaded += loaded
                print(f"   ✓ Loaded {loaded} to bronze")

            committees_processed += 1

    except Exception as e:
        print(f"   ✗ Error: {e}")
        continue

print(f"\n{'='*60}")
print(f"TOTAL: {total_loaded} contributions from {committees_processed} committees")

In [ ]:
# Cell 6: Verify Bronze Layer
print("="*60)
print("Bronze Layer Summary")
print("="*60)

with get_db_session() as session:
    committee_count = session.execute(
        select(func.count()).select_from(BronzeFECCommittee)
    ).scalar()

    candidate_count = session.execute(
        select(func.count()).select_from(BronzeFECCandidate)
    ).scalar()

    contribution_count = session.execute(
        select(func.count()).select_from(BronzeFECScheduleA)
    ).scalar()

    print(f"\n✓ Bronze Layer:")
    print(f"  Committees:    {committee_count:,}")
    print(f"  Candidates:    {candidate_count:,}")
    print(f"  Contributions: {contribution_count:,}")

In [ ]:
# Cell 6b: Transform Bronze → Silver (Committees)
print("="*60)
print("STEP 3a: Transform Bronze → Silver (Committees)")
print("="*60)

from fund_lens_etl.transformers.bronze_to_silver_entities import BronzeToSilverCommitteeTransformer
from fund_lens_etl.models.silver.fec import SilverFECCommittee

with get_db_session() as session:
    # Load bronze committees
    stmt = select(BronzeFECCommittee)
    result = session.execute(stmt)

    bronze_committees = []
    for row in result.scalars():
        bronze_committees.append({
            'committee_id': row.committee_id,
            'name': row.name,
            'committee_type': row.committee_type,
            'designation': row.designation,
            'state': row.state,
            'city': row.city,
            'zip': row.zip,
            'treasurer_name': row.treasurer_name,
            'is_active': row.is_active,
            'cycles': row.cycles,
        })

    bronze_committee_df = pd.DataFrame(bronze_committees)
    print(f"Loaded {len(bronze_committee_df)} bronze committee records")

    # Transform
    committee_transformer = BronzeToSilverCommitteeTransformer()
    silver_committee_df = committee_transformer.transform(bronze_committee_df, election_cycle=2026)

    print(f"✓ Transformed to {len(silver_committee_df)} silver committee records")

    # Insert to database
    for idx, row in silver_committee_df.iterrows():
        silver_committee = SilverFECCommittee(
            source_committee_id=row['source_committee_id'],
            name=row['name'],
            committee_type=row.get('committee_type'),
            designation=row.get('designation'),
            state=row.get('state'),
            city=row.get('city'),
            zip=row.get('zip'),
            treasurer_name=row.get('treasurer_name'),
            is_active=row.get('is_active', True),
            election_cycle=row['election_cycle'],
        )
        session.add(silver_committee)

    session.commit()
    print(f"✓ Loaded {len(silver_committee_df)} committees to silver_fec_committee")

In [ ]:
# Cell 6c: Transform Bronze → Silver (Candidates)
print("="*60)
print("STEP 3b: Transform Bronze → Silver (Candidates)")
print("="*60)

from fund_lens_etl.transformers.bronze_to_silver_entities import BronzeToSilverCandidateTransformer
from fund_lens_etl.models.silver.fec import SilverFECCandidate

with get_db_session() as session:
    # Load bronze candidates
    stmt = select(BronzeFECCandidate)
    result = session.execute(stmt)

    bronze_candidates = []
    for row in result.scalars():
        bronze_candidates.append({
            'candidate_id': row.candidate_id,
            'name': row.name,
            'office': row.office,
            'state': row.state,
            'district': row.district,
            'party': row.party,
            'is_active': row.is_active,
            'cycles': row.cycles,
        })

    bronze_candidate_df = pd.DataFrame(bronze_candidates)
    print(f"Loaded {len(bronze_candidate_df)} bronze candidate records")

    # Transform
    candidate_transformer = BronzeToSilverCandidateTransformer()
    silver_candidate_df = candidate_transformer.transform(bronze_candidate_df, election_cycle=2026)

    print(f"✓ Transformed to {len(silver_candidate_df)} silver candidate records")

    # Insert to database
    for idx, row in silver_candidate_df.iterrows():
        silver_candidate = SilverFECCandidate(
            source_candidate_id=row['source_candidate_id'],
            name=row['name'],
            office=row['office'],
            state=row['state'],
            district=row.get('district'),
            party=row.get('party'),
            is_active=row.get('is_active', True),
            election_cycle=row['election_cycle'],
        )
        session.add(silver_candidate)

    session.commit()
    print(f"✓ Loaded {len(silver_candidate_df)} candidates to silver_fec_candidate")

In [ ]:
# Cell 7: Transform Bronze → Silver
print("="*60)
print("STEP 3: Transform Bronze → Silver (with enrichment)")
print("="*60)

with get_db_session() as session:
    # Load bronze contributions
    stmt = select(BronzeFECScheduleA)
    result = session.execute(stmt)

    bronze_data = []
    for row in result.scalars():
        bronze_data.append({
            'sub_id': row.sub_id,
            'transaction_id': row.transaction_id,
            'file_number': row.file_number,
            'contribution_receipt_date': row.contribution_receipt_date,
            'contribution_receipt_amount': row.contribution_receipt_amount,
            'contributor_aggregate_ytd': row.contributor_aggregate_ytd,
            'contributor_name': row.contributor_name,
            'contributor_first_name': row.contributor_first_name,
            'contributor_last_name': row.contributor_last_name,
            'contributor_city': row.contributor_city,
            'contributor_state': row.contributor_state,
            'contributor_zip': row.contributor_zip,
            'contributor_employer': row.contributor_employer,
            'contributor_occupation': row.contributor_occupation,
            'entity_type': row.entity_type,
            'committee_id': row.committee_id,
            'recipient_committee_designation': row.recipient_committee_designation,
            'receipt_type': row.receipt_type,
            'election_type': row.election_type,
            'memo_text': row.memo_text,
            'two_year_transaction_period': row.two_year_transaction_period,
            'report_year': row.report_year,
        })

    bronze_df = pd.DataFrame(bronze_data)
    print(f"Loaded {len(bronze_df)} bronze records")

    # Transform with enrichment
    transformer = BronzeToSilverFECTransformer(session=session)
    silver_df = transformer.transform(bronze_df)

    print(f"✓ Transformed to {len(silver_df)} silver records")

    # Check enrichment
    with_committee = silver_df['committee_name'].notna().sum()
    with_candidate = silver_df['candidate_name'].notna().sum()

    print(f"\nEnrichment stats:")
    print(f"  With committee name: {with_committee}/{len(silver_df)} ({with_committee/len(silver_df)*100:.1f}%)")
    print(f"  With candidate name: {with_candidate}/{len(silver_df)} ({with_candidate/len(silver_df)*100:.1f}%)")

In [ ]:
# Cell 8: Load Silver to Database
print("="*60)
print("Load Silver Records to Database")
print("="*60)

with get_db_session() as session:
    # Insert silver records
    for idx, row in silver_df.iterrows():
        silver_record = SilverFECContribution(
            source_sub_id=row['sub_id'],
            transaction_id=row['transaction_id'],
            file_number=row.get('file_number'),
            contribution_date=row['contribution_date'],
            contribution_amount=row['contribution_amount'],
            contributor_aggregate_ytd=row.get('contributor_aggregate_ytd'),
            contributor_name=row['contributor_name'],
            contributor_first_name=row.get('contributor_first_name'),
            contributor_last_name=row.get('contributor_last_name'),
            contributor_city=row.get('contributor_city'),
            contributor_state=row.get('contributor_state'),
            contributor_zip=row.get('contributor_zip'),
            contributor_employer=row.get('contributor_employer', 'NOT PROVIDED'),
            contributor_occupation=row.get('contributor_occupation', 'NOT PROVIDED'),
            entity_type=row['entity_type'],
            committee_id=row['committee_id'],
            committee_name=row.get('committee_name'),
            committee_type=row.get('committee_type'),
            committee_designation=row.get('committee_designation'),
            committee_party=row.get('committee_party'),
            candidate_id=row.get('candidate_id'),
            candidate_name=row.get('candidate_name'),
            candidate_office=row.get('candidate_office'),
            candidate_party=row.get('candidate_party'),
            receipt_type=row.get('receipt_type'),
            election_type=row.get('election_type'),
            memo_text=row.get('memo_text'),
            election_cycle=row['election_cycle'],
            report_year=row.get('report_year'),
        )
        session.add(silver_record)

    session.commit()

    # Verify
    silver_count = session.execute(
        select(func.count()).select_from(SilverFECContribution)
    ).scalar()

    print(f"✓ Loaded {silver_count} records to silver_fec_contribution")

In [ ]:
# Cell 9: Transform Silver → Gold
print("="*60)
print("STEP 4: Transform Silver → Gold (entity resolution)")
print("="*60)

with get_db_session() as session:
    # Load silver data
    stmt = select(SilverFECContribution)
    result = session.execute(stmt)

    silver_data = []
    for row in result.scalars():
        silver_data.append({
            'sub_id': row.source_sub_id,
            'contribution_date': row.contribution_date,
            'contribution_amount': row.contribution_amount,
            'contributor_name': row.contributor_name,
            'contributor_first_name': row.contributor_first_name,
            'contributor_last_name': row.contributor_last_name,
            'contributor_city': row.contributor_city,
            'contributor_state': row.contributor_state,
            'contributor_zip': row.contributor_zip,
            'contributor_employer': row.contributor_employer,
            'contributor_occupation': row.contributor_occupation,
            'entity_type': row.entity_type,
            'committee_id': row.committee_id,
            'committee_name': row.committee_name,
            'committee_designation': row.committee_designation,
            'committee_party': row.committee_party,
            'candidate_id': row.candidate_id,
            'candidate_name': row.candidate_name,
            'candidate_office': row.candidate_office,
            'candidate_party': row.candidate_party,
            'receipt_type': row.receipt_type,
            'election_type': row.election_type,
            'election_cycle': row.election_cycle,
            'memo_text': row.memo_text,
        })

    silver_df_gold = pd.DataFrame(silver_data)
    print(f"Loaded {len(silver_df_gold)} silver records")

    # Transform to gold
    gold_transformer = SilverToGoldFECTransformer(session=session)
    gold_df = gold_transformer.transform(silver_df_gold)

    print(f"✓ Transformed to {len(gold_df)} gold records")

In [ ]:
# Cell 10: Load Gold to Database
print("="*60)
print("Load Gold Records to Database")
print("="*60)

with get_db_session() as session:
    # Insert gold contribution records
    for idx, row in gold_df.iterrows():
        gold_record = GoldContribution(
            source_system=row['source_system'],
            source_transaction_id=row['source_transaction_id'],
            contribution_date=row['contribution_date'],
            amount=row['amount'],
            contributor_id=row['contributor_id'],
            recipient_committee_id=row['recipient_committee_id'],
            recipient_candidate_id=row.get('recipient_candidate_id'),
            contribution_type=row['contribution_type'],
            election_type=row.get('election_type'),
            election_year=row['election_year'],
            election_cycle=row['election_cycle'],
            memo_text=row.get('memo_text'),
        )
        session.add(gold_record)

    session.commit()

print("✓ Gold contributions loaded")

In [ ]:
# Cell 11: Final Verification - All Layers
print("="*60)
print("FINAL PIPELINE SUMMARY")
print("="*60)

with get_db_session() as session:
    # Bronze
    bronze_committees = session.execute(
        select(func.count()).select_from(BronzeFECCommittee)
    ).scalar()
    bronze_candidates = session.execute(
        select(func.count()).select_from(BronzeFECCandidate)
    ).scalar()
    bronze_contributions = session.execute(
        select(func.count()).select_from(BronzeFECScheduleA)
    ).scalar()

    # Silver
    silver_contributions = session.execute(
        select(func.count()).select_from(SilverFECContribution)
    ).scalar()
    silver_committees = session.execute(
        select(func.count()).select_from(SilverFECCommittee)
    ).scalar()
    silver_candidates = session.execute(
        select(func.count()).select_from(SilverFECCandidate)
    ).scalar()

    # Gold
    gold_contributors = session.execute(
        select(func.count()).select_from(GoldContributor)
    ).scalar()
    gold_committees = session.execute(
        select(func.count()).select_from(GoldCommittee)
    ).scalar()
    gold_candidates = session.execute(
        select(func.count()).select_from(GoldCandidate)
    ).scalar()
    gold_contributions = session.execute(
        select(func.count()).select_from(GoldContribution)
    ).scalar()

    print("\n✓ BRONZE LAYER:")
    print(f"  Committees:    {bronze_committees:,}")
    print(f"  Candidates:    {bronze_candidates:,}")
    print(f"  Contributions: {bronze_contributions:,}")

    print("\n✓ SILVER LAYER:")
    print(f"  Committees:    {silver_committees:,}")
    print(f"  Candidates:    {silver_candidates:,}")
    print(f"  Contributions: {silver_contributions:,}")

    print("\n✓ GOLD LAYER:")
    print(f"  Contributors:  {gold_contributors:,}")
    print(f"  Committees:    {gold_committees:,}")
    print(f"  Candidates:    {gold_candidates:,}")
    print(f"  Contributions: {gold_contributions:,}")

    # Sample enriched record
    print(f"\n{'='*60}")
    print("Sample Gold Contribution with Full Context:")

    stmt = select(
        GoldContribution,
        GoldContributor,
        GoldCommittee,
        GoldCandidate
    ).join(
        GoldContributor, GoldContribution.contributor_id == GoldContributor.id
    ).join(
        GoldCommittee, GoldContribution.recipient_committee_id == GoldCommittee.id
    ).outerjoin(
        GoldCandidate, GoldContribution.recipient_candidate_id == GoldCandidate.id
    ).limit(1)

    result = session.execute(stmt).first()

    if result:
        contribution, contributor, committee, candidate = result
        print(f"\nContributor: {contributor.name}")
        print(f"  Location: {contributor.city}, {contributor.state}")
        print(f"  Employer: {contributor.employer}")
        print(f"\nContribution: ${contribution.amount}")
        print(f"  Date: {contribution.contribution_date}")
        print(f"  Type: {contribution.contribution_type}")
        print(f"\nRecipient Committee: {committee.name}")
        print(f"  Type: {committee.committee_type}")
        print(f"  Party: {committee.party}")

        if candidate:
            print(f"\nCandidate: {candidate.name}")
            print(f"  Office: {candidate.office}")
            print(f"  Party: {candidate.party}")

    print(f"\n{'='*60}")
    print("✓ END-TO-END PIPELINE COMPLETE!")